# Creating Topography Variables

In [1]:
%load_ext jupyter_black
from bs4 import BeautifulSoup
import requests
from pathlib import Path
import os
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import richdem as rd
from rasterstats import zonal_stats
import matplotlib.pyplot as plt

In [2]:
base_url = Path(os.getenv("STORM_DATA_DIR")) / "analysis/02_new_model_input/"
input_dir = base_url / "04_topography/input/srtm/"
output_dir = base_url / "04_topography/output/"
shp_output_dir = base_url / "02_housing_damage/output/"

In [3]:
grid = gpd.read_file(shp_output_dir / "phl_0.1_degree_grid_land_overlap.gpkg")

In [4]:
# reading in the files
# merging rasters
fileList = os.listdir(input_dir)
mosaic_raster = []
for file in fileList:
    rast = rasterio.open(input_dir / file)
    mosaic_raster.append(rast)

merged_raster, out_raster = merge(mosaic_raster)

In [5]:
out_meta = rast.meta.copy()
out_meta.update(
    {
        "driver": "GTiff",
        "height": merged_raster.shape[1],
        "width": merged_raster.shape[2],
        "transform": out_raster,
        "crs": grid.crs,
    }
)

In [6]:
with rasterio.open(input_dir / "phl_merged_srtm.tif", "w", **out_meta) as dest:
    dest.write(merged_raster)

In [12]:
merged_rast = rasterio.open(input_dir / "phl_merged_srtm.tif")
merged_array = merged_rast.read(1)

## Slope

In [ ]:
sp_dem = rd.rdarray(merged_raster, no_data=rast.nodata)
slope = rd.TerrainAttribute(sp_dem, attrib="slope_riserun")
rd.rdShow(slope, axes=False, cmap="magma", figsize=(20, 12))
plt.show()

### Mean Slope

### Slope Deviation

## Mean Elevation

In [16]:
# computing stats for 4 adjacent cells
# checking if crs are the same
summary_stats = zonal_stats(
    grid,
    merged_array,
    stats=["mean"],
    nodata=-32768,
    all_touched=True,
    affine=merged_rast.transform,
)
grid_elev = pd.DataFrame(summary_stats)

## Ruggedness

### Mean ruggedness

### Ruggedness deviation

## Area in Square Kilometers

## Coast

### With Coast

### Coast Length